In [ ]:
BRANCH='r1.8.0'

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

In [ ]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

In the era of super large language models, the traditional "pre-train, fine-tune" procedure is replaced by "pre-train, prompt, and predict" method as shown in the [survey paper](https://arxiv.org/pdf/2107.13586.pdf). The prompt method is versatile enough to support all kinds of NLP tasks as shown in the following table: 

<table>
    <thead>
        <tr>
            <th>Type</th>
            <th>Task</th>
            <th>Input ([X])</th>
            <th>Template</th>
            <th>Answer([Y])</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan=3>Text CLS</td>
            <td>Sentiment</td>
            <td>I love this movie.</td>
            <td>[X] The movie is [Y]</td>
            <td>great<br>fantastic<br>...</td>
        </tr>
        <tr>
            <td>Topics</td>
            <td>He prompted the LM.</td>
            <td>[X] The text is about [Y]</td>
            <td>sports<br>science<br>...</td>
        </tr>
        <tr>
            <td>Intention</td>
            <td>What is taxi fare to Denver?</td>
            <td>[X] The question is about [Y]</td>
            <td>quantity<br>city<br>...</td>
        </tr>
        <tr>
            <td rowspan=1>Text-span CLS</td>
            <td>Aspect<br>Sentiment</td>
            <td>Poor service but good food.</td>
            <td>[X] What about service? [Y]</td>
            <td>Bad<br>Terrible<br>...</td>
        </tr>
        <tr>
            <td rowspan=1>Text-pair CLS</td>
            <td>NLI</td>
            <td>[X1]: An old man with ...<br>[X2]: A man walks ...</td>
            <td>Hypothesis: [X1], Premise: [X2], Answer: [Y]</td>
            <td>Contradiction<br>Entailment<br>...</td>
        </tr>
        <tr>
            <td rowspan=1>Tagging</td>
            <td>NER</td>
            <td>[X1]: Mike went to Paris.<br>[X2]: Paris</td>
            <td>[X1] [X2] is a [Y]</td>
            <td>Yes<br>No<br>...</td>
        </tr>
        <tr>
            <td rowspan=2>Text Generation</td>
            <td>Summarization</td>
            <td>Las Vegas police ...</td>
            <td>[X] TL;DR: [Y]</td>
            <td>The victim ...<br>A woman ...<br>...</td>
        </tr>
        <tr>
            <td>Translation</td>
            <td>Je vous aime.</td>
            <td>French [X] English: [Y]</td>
            <td>I love you.<br>I fancy you.<br>...</td>
        </tr>
    </tbody>
</table>

In this tutorial, we are going to describe how to use [P-Tuning method](https://arxiv.org/pdf/2103.10385.pdf) , which is one of the prompt engineering methods, to find good prompts for large GPT models. We show it can solve multiple downstream NLP tasks with good performance. P-Tuning leverages few continuous free parameters to serve as prompts fed as the input to the pre-trained language models. Freezing the large language model weights, P-Tuning model can be trained efficiently while delivering stats of art performance. 

Large Language Model can be trained with [NeMo Megatron](https://github.com/NVIDIA/NeMo/tree/main/examples/nlp/language_modeling), up to multi-billion parameters. In this notebook, we will use the pre-trained 344M GPT model released from NGC.

# Task Description
P-Tuning method can be applied to solve various NLP tasks. Without losing generality, in this notebook, we are going to use P-Tuning method to solve two NLP tasks: **Sentiment Analysis** task and **Question and Answer** task. 

**Sentiment Analysis** task is also known as opinion mining or emotion AI. It is a sub-field of NLP that tries to identify and extract opinions within a given text across blogs, reviews, social media, forums, news etc. 

For instance, **given sentences from news title, is it a good or bad news?**<br>

**Question and Answer** task is to find the answer to a question given the context text. 

For instance, 
```
Context: 
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Question:
How many Grammy awards did Beyoncé win for her first solo album?
```

# Dataset
We will use [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) for sentiment analysis task and [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) for question and answer task.

The [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) contains the sentiments for financial news headlines from the perspective of a retail investor. Further details about the dataset can be found in: Malo, P., Sinha, A., Takala, P., Korhonen, P. and Wallenius, J. (2014): “Good debt or bad debt: Detecting semantic orientations in economic texts.” Journal of the American Society for Information Science and Technology.

Here's an example of what an annotated abstract from the corpus looks like:

```
HELSINKI Thomson Financial - Shares in Cargotec fell sharply in early afternoon trade after the cargo handling group posted a surprise drop in April-June profits , which overshadowed the large number of new orders received during the three months .@negative
LONDON MarketWatch -- Share prices ended lower in London Monday as a rebound in bank stocks failed to offset broader weakness for the FTSE 100 .@negative
Operating profit fell to EUR 35.4 mn from EUR 68.8 mn in 2007 , including vessel sales gain of EUR 12.3 mn .@negative
Sales in Finland decreased by 10.5 % in January , while sales outside Finland dropped by 17 % .@negative
```

The [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

Let's download the dataset.

In [ ]:
DATA_DIR = "DATA_DIR"
os.makedirs(DATA_DIR, exist_ok=True)

## Downloading Financial Phrase Bank Dataset

The dataset is collected by Malo et al. 2014, and can be downloaded from this [link](https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip). The zip file for the Financial Phrase Bank Dataset has been provided for ease of download and use.

In [ ]:
!wget https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip
!unzip FinancialPhraseBank-v10.zip -d {DATA_DIR}

In [ ]:
# If you want to see more examples, you can explore the text of the corpus using the file browser to the left, or open files directly, for example typing a command like the following in a code-cell:

! head -1 $DATA_DIR/FinancialPhraseBank-v1.0/Sentences_50Agree.txt

## Download the SQuAD dataset

Download a copy of the dataset (distributed under the CC BY-SA 4.0 license):

In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!mv train-v2.0.json {DATA_DIR}

## Pre-process Financial Phrase Bank Dataset

In this pre-process step, we are going to convert the downloaded dataset into the format that can be used for P-Tuning dataloader. The data is split into 10 folds so we can do 10-fold cross validation. In this notebook, we will use the first fold.

In [ ]:
import json
import random

random.seed(1234)
files = ['Sentences_50Agree.txt', 'Sentences_66Agree.txt', 'Sentences_75Agree.txt', 'Sentences_AllAgree.txt']
base_dir = DATA_DIR + '/FinancialPhraseBank-v1.0/'
files = [base_dir + f for f in files]

alllines = []
for fn in files:
    with open(fn, 'r', encoding="ISO-8859-1") as f:
        alllines.extend(f.readlines())

random.shuffle(alllines)
fold = 10
fold_size = len(alllines) // fold

chunk_start = list(range(0, 14780, 1478))

chunks = []

for start_id in chunk_start:
    chunks.append(alllines[start_id:start_id+fold_size])

def gen_file(data, fold_id, split_type):
    filename = "{}/{}_{}.txt".format(base_dir, split_type, fold_id)
    with open(filename, 'w') as f:
        obj = {}
        for line in data:
            splits = line.split('@')
            part1 = splits[0].strip()
            part2 = splits[1].strip()
            obj['sentence'] = part1
            obj['label'] = part2
            obj['taskname'] = 'sentiment-task'
            f.write(json.dumps(obj)+'\n')


def gen_fold(fold_number):
    lists = list(range(fold))
    test_id = (fold_number + fold) % fold
    val_id = (fold_number + fold - 1) % fold
    test_set = chunks[test_id]
    val_set = chunks[val_id]
    lists.remove(test_id)
    lists.remove(val_id)
    train_set = []
    for idd in lists:
        train_set += chunks[idd]
    gen_file(train_set, fold_number, 'train')
    gen_file(val_set, fold_number, 'validation')
    gen_file(test_set, fold_number, 'test')

gen_fold(0)

The data is converted to the loss json file. Each line has three keys "sentence", "label" and "prompt_tag". 
Here are the first two lines of converted data:

In [ ]:
!head -n 2 $DATA_DIR/FinancialPhraseBank-v1.0/train_0.txt

### Preprocess SQuAD Dataset


In [ ]:
file_name = DATA_DIR + '/train-v2.0.json'
with open(file_name, 'r') as f:
    data_obj = json.load(f)

articles = data_obj['data']
test_len = 40
validation_len = 40
train_len = len(articles) - test_len - validation_len
train_records = []
validation_records = []
test_records = []


def get_records(sub_articals, records):
    for article in sub_articals:
        paragraphs = article['paragraphs']
        for paragraph in paragraphs:
            qas = paragraph['qas']
            context = paragraph['context'].strip()
            for qa in qas:
                record = {}
                record['question'] = qa['question'].strip()
                record['context'] = context
                if qa['is_impossible']:
                    record['label'] = 'NA'
                else:
                    record['label'] = qa['answers'][0]['text'].strip()
                record['taskname'] = 'qa-task'
                records.append(json.dumps(record))
get_records(articles[:train_len], train_records)
get_records(articles[train_len:train_len+validation_len], validation_records)
get_records(articles[train_len+validation_len:], test_records)
random.shuffle(train_records)
random.shuffle(validation_records)
random.shuffle(test_records)
squad_dir = "DATA_DIR/squad"
os.makedirs(squad_dir, exist_ok=True)
with open(squad_dir+'/train.txt', 'w') as f:
    f.write("\n".join(train_records))
with open(squad_dir+'/validation.txt', 'w') as f:
    f.write("\n".join(validation_records))
with open(squad_dir+'/test.txt', 'w') as f:
    f.write("\n".join(test_records))



The data is converted to the loss json file. Each line has three keys "question", "context", "label" and "prompt_tag". 
Here are the first two lines of converted data:

In [ ]:
!head -n 2 {squad_dir}/train.txt

### Combine the two datasets

The P-tune model includes a prompt encoder which is used to generate virtual tokens. Its output can be conditioned on the task tags so the P-tune model supports multiple tasks simultaneously. We are going to mix the Financial phrase bank dataset and SQuAD dataset together.

In [ ]:
mix_data_dir = f"{DATA_DIR}/mix"
os.makedirs(mix_data_dir, exist_ok=True)
!cat $DATA_DIR/FinancialPhraseBank-v1.0/train_0.txt {squad_dir}/train.txt | shuf > {mix_data_dir}/train.txt
!cat $DATA_DIR/FinancialPhraseBank-v1.0/validation_0.txt {squad_dir}/validation.txt | shuf > {mix_data_dir}/validation.txt
!cat $DATA_DIR/FinancialPhraseBank-v1.0/test_0.txt {squad_dir}/test.txt | shuf > {mix_data_dir}/test.txt

Here are the first two lines of converted data:

In [ ]:
!head -n 2 {mix_data_dir}/train.txt

## Convert the Megatron-LM Weights to Nemo file

P-Tuning method works the best with large GPT language models. From our experiences, models of size 5B or above give good performance. If you already have a large GPT model ready, skip this section. 

In this example, we will use the pretrained 344M NeMo Megatron GPT model from [Megatron-LM project](https://github.com/NVIDIA/Megatron-LM). To load it in NeMo Megatron, We first need to convert the Megatron-LM checkpoint to the `.nemo` file. Let's download the pretrained model weights and vocabulary file.



In [ ]:
import pathlib
gpt_file = 'megatron_lm_345m_v0.0.zip'
vocab_file = 'gpt2-vocab.json'
merge_file = 'gpt2-merge.txt'
checkpoint_filename = 'model_optim_rng.pt'

if not pathlib.Path(gpt_file).exists():
    !wget --content-disposition https://api.ngc.nvidia.com/v2/models/nvidia/megatron_lm_345m/versions/v0.0/zip -O $gpt_file
    !unzip -o $gpt_file
    !wget https://s3.amazonaws.com/models.huggingface.co/bert/$vocab_file -O $vocab_file 
    !wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt -O $merge_file



In [ ]:
WORK_DIR = "WORK_DIR"
os.makedirs(WORK_DIR, exist_ok=True)

# Prepare the model parameters 
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
MODEL_CONFIG = "megatron_gpt_config.yaml"
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

In [ ]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
config.model.num_layers = 24
config.model.hidden_size = 1024
config.model.ffn_hidden_size = 4096
config.model.num_attention_heads = 16
config.model.tokenizer.vocab_file = vocab_file
config.model.tokenizer.merge_file = merge_file
config.model.tensor_model_parallel_size = 1
config.model.data.data_prefix = ''
config.model.max_position_embeddings = 1024
config.model.data.seq_length = 1024
config.model.encoder_seq_length = 1024
config.cfg = {}
config.cfg.cfg = config.model
with open('hparams.yaml', 'w') as f:
    f.write(OmegaConf.to_yaml(config.cfg))

In [ ]:
import os
PWD = os.getcwd()
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/megatron_lm_ckpt_to_nemo.py')
!python -m torch.distributed.run --nproc_per_node=1 megatron_lm_ckpt_to_nemo.py --checkpoint_folder=$PWD/release/mp_rank_00/ --checkpoint_name=$checkpoint_filename --hparams_file=$PWD/hparams.yaml --nemo_file_path=$PWD/gpt_344m.nemo --model_type=gpt --tensor_model_parallel_size=1

# Model configuration

Our P-Tuning text classification model is comprised of the pretrained GPT LM model followed by a prompt encoder layer.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

In [ ]:
MODEL_CONFIG = "megatron_gpt_prompt_learning_config.yaml"

In [ ]:
# download the model's configuration file 

config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

In [ ]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
# Note: these are small batch-sizes - increase as appropriate to available GPU capacity
config.model.batch_size = 8

# Model Training
## Setting up Data within the config

Among other things, the config file contains dictionaries called train_ds, validation_ds and test_ds. These are configurations used to setup the Dataset.


In [ ]:
# in this tutorial train and dev datasets are located in the same folder, so it is enough to add the path of the data directory to the config
config.model.data.train_ds = [DATA_DIR+'/mix/train.txt',]
config.model.data.validation_ds = [DATA_DIR+'/mix/validation.txt',]
config.model.data.test_ds = [DATA_DIR+'/mix/test.txt',]

## Add the Data Processors to Generate the Prompts

To customize different prompts for different tasks, we simply need to specify the prompt task template in the config file. The virtual token markers `<|VIRTUAL_PROMPT_#|>` signify where you want virtual tokens to be placed in the the template string. `<|VIRTUAL_PROMPT_0|>`, `<|VIRTUAL_PROMPT_1|>`, and `<|VIRTUAL_PROMPT_2|>` indicate where a number of virtual tokens matching the values given at `virtual_token_splits[0]`, `virtual_token_splits[1]` and `virtual_token_splits[2]` will be placed. The other variable fields `{var}` refer to the variables in the data record. For example:

Given the data record **{"sentence1": "And he said, Mama, I'm home.", "sentence2": "He didn't say a word."}**, along with `virtual_token_splits = [3, 3, 3]` and `prompt_template = "<|VIRTUAL_PROMPT_0|> Hypothesis: [sentence1], <|VIRTUAL_PROMPT_1|> Premise: [sentence2] <|VIRTUAL_PROMPT_2|> Answer:"`, the input will be translated into **<span style="color:red">VVV</span> Hypothesis: And he said, Mama, I'm home.<span style="color:red">VVV</span> Premise: He didn't say a word.<span style="color:red">VVV</span> Answer:**, where <span style="color:red">VVV</span> are three virtual tokens.

Let's configure the proper template for the two dataset we prepared:

In [ ]:
  config.model.task_templates = [
    {
      "taskname": "qa-task",
      "prompt_template": "<|VIRTUAL_PROMPT_0|> Context: {context} <|VIRTUAL_PROMPT_1|> Question: {question}? <|VIRTUAL_PROMPT_2|> Answer: {label}",
      "total_virtual_tokens": 9,
      "virtual_token_splits":[3, 3, 3],
      "truncate_field": "content",
    },
    {
      "taskname": "sentiment-task",
      "prompt_template": "<|VIRTUAL_PROMPT_0|> Sentence: {sentence} <|VIRTUAL_PROMPT_1|> Sentiment: {label}",
      "total_virtual_tokens": 9,
      "virtual_token_splits":[6, 3],
      "truncate_field": "sentence",
    },
  ]

Note each `task_template` item has 5 fields. Besides the `prompt_template` string, the `taskname` refers to the `taskname` in the data record. The `truncate_field` specifies which field in the data is going to be cut if the length of the input exceeds the maximum sequence length of the model.`total_virtual_tokens` specifies the total number of virtual tokens that will be inserted into the model prompt. `virtual_token_splits` specifies the number of virtual tokens that belong at each `<|VIRTUAL_PROMPT_#|>` marker. `virtual_token_splits` values should add up to `total_virtual_tokens`. The number of `virtual_token_splits` should match the number of `<|VIRTUAL_PROMPT_#|>` markers. 

After you p-tune your model this time, you can always go back and p-tune your model on more tasks without over writting the virtual prompts who've trained this time. You can also use a different number of `total_virtual_tokens` between each training session as long as tasks ptuned at the same time have the same number of `total_virtual_tokens`. For this reason, you ptune on a new task, you need to tell your model which of your tasks are new and which ones already exist (and thus you don't want to tune them). 

You do this by setting the `new_tasks` and `existing_tasks` values in the config file. Because we are ptuning a model with no existing tasks, you should set `existing_tasks=[]` and `new_tasks=['qa-task', 'sentiment-task']` as follows:

In [ ]:
config.model.new_tasks = ['qa-task', 'sentiment-task']
config.model.existing_tasks = []

After ptuning is complete, you can run inference on all tasks at the same time, regradless of their `total_virtual_tokens` value.

In [ ]:
print(OmegaConf.to_yaml(config.model))

## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPPlugin
from pytorch_lightning.plugins.environments.torchelastic_environment import TorchElasticEnvironment

# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.accelerator = accelerator
config.trainer.devices = 1
config.trainer.max_epochs = 12
config.trainer.val_check_interval = 1.0

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# remove distributed training flags
config.trainer.strategy = None
plugin = NLPDDPPlugin()

# setup cluster environment parameters
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'
# use torch elastic cluster environment so `create_process_externally` is True
# the launcher is set to None. It will not try to spawn new processes.
# It won't create the misconfiguration error because of the `interactive session`
trainer = pl.Trainer(plugins=[plugin, TorchElasticEnvironment()], **config.trainer)

## Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [ ]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
os.makedirs(WORK_DIR, exist_ok=True)

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

We will use the converted `.nemo` file as our LM model.

In [ ]:
# add the specified above model parameters to the config
config.model.language_model_path = 'gpt_344m.nemo'
config.model.tensor_model_parallel_size = 1
config.exp_manager.checkpoint_callback_params.save_top_k = 1

# Set some of the learning parameters
config.model.optim.lr = 5e-6

Your final config looks like:

In [ ]:
print("Model config - \n")
print(OmegaConf.to_yaml(config))

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.

In [ ]:
from nemo.collections.nlp.models.language_modeling.megatron_gpt_prompt_learning_model import MegatronGPTPromptLearningModel
model_ptune = MegatronGPTPromptLearningModel(cfg=config.model, trainer=trainer)

## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.
If you're not using Colab, refer to [https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks) if you're facing issues with running the cell below.

In [ ]:
try:
    from google import colab
    COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
    COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
    %load_ext tensorboard
    %tensorboard --logdir {exp_dir}
else:
    print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
# start model training
trainer.fit(model_ptune)

# Inference

To see how the model performs, we can run model in the inference mode

In [ ]:
# let's first select a subset of our dev data
query_examples = [
    {"taskname": "sentiment-task", "sentence": "The Finland-based company says it will move into an existing 260,000-square-foot facility in September ."},
    {"taskname": "qa-task", "question": "What are the closest relatives of the deinonychosaurs?", "context": "The consensus view in contemporary paleontology is that the flying theropods, or avialans, are the closest relatives of the deinonychosaurs, which include dromaeosaurids and troodontids. Together, these form a group called Paraves. Some basal members of this group, such as Microraptor, have features which may have enabled them to glide or fly. The most basal deinonychosaurs were very small. This evidence raises the possibility that the ancestor of all paravians may have been arboreal, have been able to glide, or both. Unlike Archaeopteryx and the non-avialan feathered dinosaurs, who primarily ate meat, recent studies suggest that the first avialans were omnivores."}
]
response = model_ptune.generate(inputs=query_examples, length_params=None)

print('The prediction results of some sample queries with the trained model:')
for result in response['sentences']:
    print(result)
    print("-" * 30)

## Training Script

If you have NeMo installed locally, you can also train the model with `examples/nlp/language_modeling/megatron_gpt_prompt_learning.py`.

To run training script, first change the values in `examples/nlp/language_modeling/conf/megatron_gpt_prompt_learning_config.yaml` to your desired values, then run:
```
python examples/nlp/language_modeling/megatron_gpt_prompt_learning.py \
    --config-name=prompt_learning_megatron_gpt_inference.yaml
```